In [41]:
import pandas as pd
import plotly.graph_objects as go

In [42]:
ma_test_res = pd.read_pickle("ma_test_res.pkl")
all_trades = pd.read_pickle("all_trades.pkl")

In [43]:
all_trades.shape

(65303, 11)

In [44]:
ma_test_res.head()

,pair,num_trades,total_gain,mean_gain,min_gain,max_gain,mashort,malong
0,GBP_USD,142,127.5,0.897887,-242.7,492.7,16,32
1,GBP_USD,202,-1358.4,-6.724752,-222.7,422.0,4,32
2,GBP_USD,158,-347.0,-2.196203,-222.7,507.4,8,32
3,GBP_USD,157,-199.7,-1.271975,-270.7,474.7,24,32
4,GBP_USD,16,733.6,45.850000,-125.1,488.2,32,256


In [45]:
ma_test_res = ma_test_res[['pair', 'num_trades', 'total_gain', 'mashort', 'malong']]
ma_test_res["CROSS"] = "MA_" + ma_test_res.mashort.map(str) + "_" + ma_test_res.malong.map(str)

In [46]:
df_all_gains = ma_test_res.groupby(by=["CROSS","mashort","malong"], as_index=False).sum()
df_all_gains.sort_values(by="total_gain", ascending=False, inplace=True)
df_all_gains.head()

,CROSS,mashort,malong,num_trades,total_gain
27,MA_8_32,8,32,3119,8281.9
21,MA_64_128,64,128,637,6013.3
2,MA_16_32,16,32,2764,5075.8
12,MA_32_64,32,64,1348,4377.9
10,MA_32_128,32,128,766,3887.0


In [47]:
ma_8_32 = ma_test_res[ma_test_res.CROSS == "MA_8_32"].copy()

In [48]:
ma_8_32.sort_values(by="total_gain", ascending=False, inplace=True)
ma_8_32.head()

,pair,num_trades,total_gain,mashort,malong,CROSS
362,USD_JPY,134,2701.4,8,32,MA_8_32
242,EUR_JPY,147,2688.7,8,32,MA_8_32
602,CHF_JPY,141,2044.2,8,32,MA_8_32
422,CAD_JPY,137,1524.0,8,32,MA_8_32
62,GBP_JPY,154,1255.9,8,32,MA_8_32


In [49]:
ma_8_32[ma_8_32.total_gain > 0].shape[0]

13

In [50]:
for cross in df_all_gains.CROSS.unique():
    df_temp = ma_test_res[ma_test_res.CROSS==cross]
    total_p = df_temp.shape[0]
    pos = df_temp[df_temp.total_gain>0].shape[0]
    print(f"{cross:12} {pos:4} {(pos / total_p)*100:4.0f}%")
    

MA_8_32        13   62%
MA_64_128      14   67%
MA_16_32       11   52%
MA_32_64       11   52%
MA_32_128      13   62%
MA_32_96       14   67%
MA_64_256      13   62%
MA_24_128      11   52%
MA_24_96       12   57%
MA_16_64       12   57%
MA_16_96       12   57%
MA_16_128       9   43%
MA_64_96       11   52%
MA_24_64       11   52%
MA_32_256      10   48%
MA_8_256       11   52%
MA_8_96         9   43%
MA_8_64         8   38%
MA_24_256       9   43%
MA_4_32         7   33%
MA_16_256      11   52%
MA_8_128        8   38%
MA_24_32       10   48%
MA_4_128        6   29%
MA_4_96         8   38%
MA_4_256        8   38%
MA_4_64         7   33%
MA_8_16         6   29%
MA_4_8          4   19%
MA_4_16         5   24%


In [51]:
crosses = df_all_gains.CROSS.unique()[:3]
crosses
ma_test_res.head(2)

,pair,num_trades,total_gain,mashort,malong,CROSS
0,GBP_USD,142,127.5,16,32,MA_16_32
1,GBP_USD,202,-1358.4,4,32,MA_4_32


In [52]:
df_good = ma_test_res[(ma_test_res.CROSS.isin(crosses)) & (ma_test_res.total_gain>0)].copy()
df_good.head(2)

,pair,num_trades,total_gain,mashort,malong,CROSS
0,GBP_USD,142,127.5,16,32,MA_16_32
22,GBP_USD,35,430.9,64,128,MA_64_128


In [53]:
our_pairs = list(df_good.pair.value_counts()[:9].index)

In [54]:
all_trades["CROSS"] = "MA_" + all_trades.MASHORT.map(str) + "_" + all_trades.MALONG.map(str)

In [55]:
trades_cad_jpy = all_trades[(all_trades.CROSS=="MA_8_16") & (all_trades.PAIR=="CAD_JPY")].copy()
trades_cad_jpy['CUM_GAIN'] = trades_cad_jpy.GAIN.cumsum()
trades_cad_jpy.head()

,time,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,PAIR,MASHORT,MALONG,DURATION,CROSS,CUM_GAIN
16,2022-01-19 19:00:00+00:00,91.466,-0.013125,0.016875,-1,15.6,-15.6,CAD_JPY,8,16,10.0,MA_8_16,-15.6
26,2022-01-20 05:00:00+00:00,91.622,0.010812,-0.032313,1,-16.2,-16.2,CAD_JPY,8,16,9.0,MA_8_16,-31.8
35,2022-01-20 14:00:00+00:00,91.460,-0.005812,0.019313,-1,-75.4,75.4,CAD_JPY,8,16,24.0,MA_8_16,43.6
59,2022-01-21 14:00:00+00:00,90.706,0.005938,-0.004313,1,-2.1,-2.1,CAD_JPY,8,16,2.0,MA_8_16,41.5
61,2022-01-21 16:00:00+00:00,90.685,-0.028938,0.002938,-1,-13.3,13.3,CAD_JPY,8,16,61.0,MA_8_16,54.8


In [56]:
def plot_line(df_plot, name):
    fig = go.Figure()
    fig.add_trace(go.Scatter( 
        x=df_plot.time,
        y=df_plot.CUM_GAIN,
        line=dict(width=2,color="#3d825f"),
        line_shape='spline',
        name=name,
        mode='lines'
        ))
    fig.update_layout(width=1000,height=400,
        margin=dict(l=15,r=15,b=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e",
        title=name)
    fig.update_xaxes(
        linewidth=1,
        linecolor='#3a4a54',
        showgrid=False,
        zeroline=False
    )
    fig.update_yaxes(
        linewidth=1,
        linecolor='#3a4a54',
        showgrid=False,
        zeroline=False
    )
    fig.show()

In [57]:
plot_line(trades_cad_jpy, "CAD_JPY")

In [58]:
c = 'MA_8_16'
for p in our_pairs:
    temp_df = all_trades[(all_trades.CROSS==c)&(all_trades.PAIR==p)].copy()
    temp_df['CUM_GAIN'] = temp_df.GAIN.cumsum()
    plot_line(temp_df, p + "_" + c)

In [59]:
for c in crosses:   
    temp_df = all_trades[(all_trades.CROSS==c)].copy()    
    temp_df = temp_df.groupby(by="time", as_index=False).sum()
    temp_df['CUM_GAIN'] = temp_df.GAIN.cumsum()
    plot_line(temp_df, c)